In [1]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


In [2]:
def load_documents(path="data/"):
    docs = []
    ids = []

    for filename in os.listdir(path):
        if filename.endswith(".txt"):
            with open(os.path.join(path, filename), "r", encoding="utf8") as f:
                docs.append(f.read())
                ids.append(filename)

    return pd.DataFrame({"doc_id": ids, "text": docs})


In [3]:
import nltk

In [4]:
def load_documents(path="data/"):
    docs = []
    ids = []

    for filename in os.listdir(path):
        if filename.endswith(".txt"):
            with open(os.path.join(path, filename), "r", encoding="utf8") as f:
                docs.append(f.read())
                ids.append(filename)

    return pd.DataFrame({"doc_id": ids, "text": docs})


In [5]:
import nltk
nltk.download('stopwords', quiet=True)


True

In [6]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


In [7]:
import nltk

# Download the stopwords corpus (no GUI)
nltk.download('stopwords', quiet=True)

# Then use it
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print(stop_words)


{'each', 'you', 'can', "we've", 'same', 'y', "we're", 'should', 'they', "hasn't", 'until', "isn't", 'both', "i've", 'for', "he's", 'his', "i'd", "that'll", 'ourselves', 'further', "i'm", 'this', 'd', 'before', 'when', 'a', 'will', 'was', 'themselves', 'how', 're', 'i', 'hers', 'there', "they'll", 'not', 'which', 'off', 'into', 'me', "doesn't", 'with', 's', "you've", 'any', 'did', 'hadn', 'such', 'yours', 'as', 'theirs', 'is', 'own', 'what', 'up', 'were', 'then', 'and', 'has', 'because', 'from', 'himself', "should've", 'aren', "we'd", "they've", 'have', "i'll", 'its', 'does', 'do', 'been', 'being', 'during', 'but', 'weren', 'your', 'mightn', 'the', 'of', 'an', 'myself', 'haven', 'or', 'against', 'are', 'very', "won't", "mustn't", 'in', 'having', "you'd", "she's", 'where', 'under', "aren't", 'between', 'hasn', "they're", 'll', "weren't", "haven't", "mightn't", 'won', 'through', 'her', "couldn't", 'wouldn', 'than', 'over', "he'll", 'doesn', 'so', 'out', "you're", 'we', 'some', 'yourselves

In [9]:
docs = []
ids = []



In [8]:
def load_documents(path="data/"):
    """
    This function loads all .txt files from the specified folder
    and returns a DataFrame containing file names and text content.
    """
    docs = []  # List to store the content of each document
    ids = []   # List to store the filenames (document IDs)

    # Loop through all files in the specified folder
    for filename in os.listdir(path):
        if filename.endswith(".txt"):  # Only process text files
            with open(os.path.join(path, filename), "r", encoding="utf8") as f:
                docs.append(f.read())     # Read and store file content
                ids.append(filename)      # Store the filename

    # Return a DataFrame with two columns: doc_id and text
    return pd.DataFrame({"doc_id": ids, "text": docs})



In [8]:
import os

path = "C:/projects/ir-system/data/"  # Use forward slashes
print("Folder exists:", os.path.exists(path))
print("Files in folder:", os.listdir(path))  # Lists files if folder exists


Folder exists: True
Files in folder: ['file1.txt.txt', 'file2.txt.txt']


In [11]:
import os
os.getcwd()


'C:\\Users\\SYED ZAIN UL AIDEEN'

In [9]:
os.makedirs("data", exist_ok=True)


In [13]:
import os

path = "C:/projects/ir-system/data/"  # Use forward slashes
print("Folder exists:", os.path.exists(path))
print("Files in folder:", os.listdir(path))  # Lists files if folder exists


Folder exists: True
Files in folder: ['file1.txt.txt', 'file2.txt.txt']


In [14]:
df = load_documents("C:/projects/ir-system/data/")
df.head()  # Shows the first few loaded documents


,doc_id,text
0,file1.txt.txt,
1,file2.txt.txt,


In [16]:
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [ps.stem(t) for t in tokens]
    return " ".join(tokens)


In [17]:
def preprocess_all(df):
    df["clean_text"] = df["text"].apply(preprocess)
    return df


In [18]:
def build_tfidf_index(docs):
    vectorizer = TfidfVectorizer()
    matrix = vectorizer.fit_transform(docs)
    return vectorizer, matrix


In [19]:
def build_bm25_index(tokenized_docs):
    return BM25Okapi(tokenized_docs)


In [20]:
def retrieve_tfidf(query, vectorizer, matrix, df, top_k=5):
    q_clean = preprocess(query)
    q_vec = vectorizer.transform([q_clean])
    scores = cosine_similarity(q_vec, matrix)[0]
    top_idx = scores.argsort()[::-1][:top_k]
    return df.iloc[top_idx][["doc_id", "text"]]


In [21]:
def retrieve_bm25(query, bm25, df, top_k=5):
    q_clean = preprocess(query)
    q_tokens = q_clean.split()
    scores = bm25.get_scores(q_tokens)
    top_idx = np.argsort(scores)[::-1][:top_k]
    return df.iloc[top_idx][["doc_id", "text"]]


In [22]:
def precision_at_k(retrieved, relevant, k=5):
    retrieved_k = retrieved[:k]
    return len(set(retrieved_k) & set(relevant)) / k


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np



In [26]:
df = load_documents()  # Your DataFrame with 'doc_id' and 'text'


In [28]:
def retrieve_tfidf(query, vectorizer, tfidf_matrix, df, top_n=5):
    """
    Retrieve top_n documents most relevant to the query using TF-IDF cosine similarity
    """
    # Transform the query into TF-IDF vector
    query_vec = vectorizer.transform([query])
    
    # Compute cosine similarity
    from sklearn.metrics.pairwise import cosine_similarity
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    
    # Get top_n document indices
    top_indices = similarities.argsort()[::-1][:top_n]
    
    # Return top_n documents
    return df.iloc[top_indices]


In [3]:
top_docs['text'].apply(lambda x: x[:200])


NameError: name 'top_docs' is not defined